Load API Keys

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

Load Contents from HLB Fixed Deposite Pages

In [2]:
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader([
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit.html?icp=hlb-en-all-footer-txt-fd",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/fixed-deposit-account.html", 
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/e-fixed-deposit.html",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/flexi-fd.html",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/senior-savers-flexi-fd.html",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/junior-fixed-deposit.html",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/foreign-fixed-deposit-account.html",
  "https://www.hlb.com.my/en/personal-banking/help-support/fees-and-charges/deposits.html",
])
data = loader.load()
data

[Document(page_content="\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nFixed Deposit - HLB\n\n\n\n\n\n\n\n\n\n\nFixed Deposit - HLB\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nPERSONAL\n\n\nBUSINESS\n\n\n\n                            Priority\n\n\nPriority\nIslamic\nGlobal Markets\nBlue Header\nInterim Homepage\nPriority Banking Form\ntest\ntest\nLoan Cal\nLog Viewer\nasdsadsad\nstand alone page\nMaster Form Page\nMaster Form Page\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nsingapore\n\n\n\nhong kong\n\n\n\nvietnam\n\n\n\ncambodia\n\n\n\n\n\n                            ENG\n\n\nLAN A\nLAN B\nLAN C\n\n\n\n\n                            Online Banking\n                        \n\nPersonal\nBusiness\n\n\n\n\n\n\n\n\n\n\n\n\n\n                                About Us\n                            \n\nOur Story\nNews & Updates\nInvestor Relations\nCSR\nAwards\nManagement & BODs\nCareer\n\n\n\n\n                                Discover Our Products\n      

Split the Loaded Data into Chunks

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
texts = text_splitter.split_documents(data)
texts

[Document(page_content='Fixed Deposit - HLB\n\n\n\n\n\n\n\n\n\n\nFixed Deposit - HLB\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nPERSONAL\n\n\nBUSINESS\n\n\n\n                            Priority\n\n\nPriority\nIslamic\nGlobal Markets\nBlue Header\nInterim Homepage\nPriority Banking Form\ntest\ntest\nLoan Cal\nLog Viewer\nasdsadsad\nstand alone page\nMaster Form Page\nMaster Form Page\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nsingapore\n\n\n\nhong kong\n\n\n\nvietnam\n\n\n\ncambodia\n\n\n\n\n\n                            ENG\n\n\nLAN A\nLAN B\nLAN C\n\n\n\n\n                            Online Banking\n                        \n\nPersonal\nBusiness\n\n\n\n\n\n\n\n\n\n\n\n\n\n                                About Us\n                            \n\nOur Story\nNews & Updates\nInvestor Relations\nCSR\nAwards\nManagement & BODs\nCareer\n\n\n\n\n                                Discover Our Products\n                            \n\nDeposits\nCredit Card\nLoans\nInsurance\nInvestment\nRemittance\

Initialize Pinecone

In [4]:
import pinecone

pinecone.init(
  api_key=os.getenv("PINECONE_API_KEY"),
  environment=os.getenv("PINECONE_ENVIRONMENT")
)
index_name = "main-index"
namespace = "hlb-fixed-deposit-with-conversation-chain-gpt3.5"
if index_name not in pinecone.list_indexes():
  pinecone.create_index(
    name=index_name,
    metric="cosine",
    dimension=1536
  )
index = pinecone.Index(index_name)
index.delete(delete_all=True, namespace=namespace)

d:\Code\langchain-test\venv\Lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


{}

Create Embeddings and Vectors to Insert to Pinecone Database

In [5]:
from langchain.vectorstores import Pinecone
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
# text = [t.page_content for t in texts]
vector = Pinecone.from_documents(texts, embeddings, index_name=index_name, namespace=namespace)

Create Retriever for Vector Search

In [6]:
retriever = vector.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.9, "k": 15})
retriever

VectorStoreRetriever(tags=['Pinecone', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.pinecone.Pinecone object at 0x00000286ADB2B990>, search_type='similarity_score_threshold', search_kwargs={'score_threshold': 0.9, 'k': 15})

Creating Prompt Template

In [21]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0)
prompt_template = """
  You are a helpful assistant who should reply to inputs related only to the HongLeong Bank aka HLB.
  You answer to the question: {question}
  By only referring the follwing document: {context}
  If you don't have enought information from the document, just say you don't have that information.
  Your answer should be detailed but if you have to do calculation, don't show your work or equations, double check your answer, just provide the final value, and you don't need to show reference.
  If the bank's name is not provided in the question, answer with the assumption that the question is about HLB.
  If the bank's name is provided and it's not HLB, don't answer that question.
  You don't need to answer if the question requires you to do something creative such as writing a song or making a joke.
"""
prompt = PromptTemplate.from_template(prompt_template)

Creating Memory

In [22]:
from langchain.memory import ConversationSummaryBufferMemory

memory = ConversationSummaryBufferMemory(llm=llm, memory_key="chat_history", output_key="answer", max_token_limit="500", return_messages=True)

Creating Chain

In [23]:
from langchain.chains import ConversationalRetrievalChain
from langchain.callbacks import get_openai_callback

chain = ConversationalRetrievalChain.from_llm(
  llm=llm,
  chain_type="stuff",
  retriever=retriever,
  memory=memory,
  combine_docs_chain_kwargs={"prompt": prompt},
  verbose=True,
  return_source_documents=True
)

def ask_question(question):
  with get_openai_callback() as cb:
    result = chain(question)
    print(f'{cb}\n')
    print(f'\nAnswer: {result["answer"]}')
    for source_document in result["source_documents"]:
      print(f'\Documents: {source_document}')


Running the Chain

In [18]:
ask_question("What is fixed deposit?")



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: How many types of fixed deposit does HongLeong Bank provide?
Assistant: HongLeong Bank provides several types of fixed deposit accounts. The types of fixed deposit accounts offered by HongLeong Bank are:

1. Junior Fixed Deposit: This account is designed for children below the age of 18 and offers attractive interest rates. The minimum placement for this account starts from RM1,000.

2. e-Fixed Deposit: This account allows you to manage your fixed deposits online through Hong Leong Connect. It offers attractive interest rates and provides the flexibility of early partial withdrawal without losing interest on the remaining balance.

3. Senior Savers Flexi FD: This account is designed for individuals aged 50 years and above. It offers attractive interest rates an

In [25]:
ask_question("How many types of fixed deposit does HongLeong Bank provide?")



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: How many types of fixed deposit does HongLeong Bank provide?
Assistant: HongLeong Bank provides several types of fixed deposit accounts, including:

1. Junior Fixed Deposit: Designed for children below the age of 18, with attractive interest rates and a minimum placement of RM1,000.

2. Senior Savers Flexi Fixed Deposit: Designed for individuals aged 50 years and above, offering attractive interest rates and allowing partial withdrawals without losing the FD interest on the remaining balance.

3. e-Fixed Deposit: Offers attractive interest rates, credited at maturity into any Hong Leong Savings or Current Account, and allows for early fixed deposit partial withdrawals without losing interest on the remaining balance.

4. Foreign Currency Fixed Deposit: Provides

In [ ]:
ask_question("What is the difference between Fixed Deposit and eFixed Deposit?")

In [20]:
ask_question("What are the interest rates for Fixed Deposit?")



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

system: The human asks how many types of fixed deposit accounts HongLeong Bank provides. The AI responds by listing several types of fixed deposit accounts offered by HongLeong Bank, including Junior Fixed Deposit, e-Fixed Deposit, Senior Savers Flexi FD, Foreign Currency Fixed Deposit, and Fixed Deposit. The AI also advises checking with the bank for the most up-to-date information on interest rates and terms.
Human: What is fixed deposit?
Assistant: A fixed deposit is a type of savings account offered by Hong Leong Bank (HLB) where you can deposit a specific amount of money for a fixed period of time, known as the tenure. The interest rate for the fixed deposit is determined by HLB and is usually higher than the interest rate for a regular savings account. The inter

In [19]:
ask_question("Lets say I want to invest RM 50,000 in Fixed Deposit for 12 months. Please calculate the total amount that I can withdraw  at the end of the term.")



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: How many types of fixed deposit does HongLeong Bank provide?
Assistant: HongLeong Bank provides several types of fixed deposit accounts. The types of fixed deposit accounts offered by HongLeong Bank are:

1. Junior Fixed Deposit: This account is designed for children below the age of 18 and offers attractive interest rates. The minimum placement for this account starts from RM1,000.

2. e-Fixed Deposit: This account allows you to manage your fixed deposits online through Hong Leong Connect. It offers attractive interest rates and provides the flexibility of early partial withdrawal without losing interest on the remaining balance.

3. Senior Savers Flexi FD: This account is designed for individuals aged 50 years and above. It offers attractive interest rates an